# Problem Statement : Hospital Patient Data Analysis

A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.

# 1.Load the patient dataset and show summary with info().

In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/Python excelr/Patient_Data.csv.xls")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


# 2.Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [5]:
billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
df_billing = df[billing_cols]
print("\nBilling-relevant columns:\n", df_billing.head())


Billing-relevant columns:
    PatientID   Department     Doctor  BillAmount
0        101   Cardiology  Dr. Smith      5000.0
1        102    Neurology   Dr. John         NaN
2        103  Orthopedics    Dr. Lee      7500.0
3        104   Cardiology  Dr. Smith      6200.0
4        105  Dermatology   Dr. Rose         NaN


# 3.Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [6]:
admin_cols = ['ReceptionistID', 'CheckInTime']
df_cleaned = df.drop(columns=admin_cols, errors='ignore')
print("\nDataset after dropping administrative columns:\n", df_cleaned.head())


Dataset after dropping administrative columns:
    PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


# 4.Use groupby to find total bill amount per department.

In [7]:
df_cleaned['BillAmount'] = pd.to_numeric(df_cleaned['BillAmount'], errors='coerce')
dept_total_bill = df.groupby('Department')['BillAmount'].sum().reset_index()
print("\nTotal bill amount per department:\n", dept_total_bill)


Total bill amount per department:
     Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


# 5.Remove duplicate patient records based on PatientID.

In [9]:
df_unique = df_cleaned.drop_duplicates(subset='PatientID', keep='first')
print("\nDataset after removing duplicate PatientID:\n", df_unique.head())


Dataset after removing duplicate PatientID:
    PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN


# 6.Fill missing BillAmount values with the mean bill amount.

In [10]:
mean_bill = df_unique['BillAmount'].mean()
df_unique['BillAmount'] = df_unique['BillAmount'].fillna(mean_bill)
print("\nDataset after filling missing BillAmount:\n", df_unique.head())


Dataset after filling missing BillAmount:
    PatientID     Name   Department     Doctor   BillAmount
0        101    Alice   Cardiology  Dr. Smith  5000.000000
1        102      Bob    Neurology   Dr. John  6233.333333
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000
3        104    David   Cardiology  Dr. Smith  6200.000000
4        105      Eva  Dermatology   Dr. Rose  6233.333333


/tmp/ipython-input-3685631408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['BillAmount'] = df_unique['BillAmount'].fillna(mean_bill)


# 7.Merge the billing dataset with patient dataset on PatientID.

In [11]:
billing_data = pd.read_csv('/content/drive/MyDrive/Python excelr/Billing_Data.csv.xls')
merged_data = pd.merge(df_unique, billing_data, on='PatientID', how='left')
print("\nMerged dataset:\n", merged_data.head())


Merged dataset:
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  


# 8.Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [12]:
new_patients = pd.DataFrame({
    'PatientID': [101, 102],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. A', 'Dr. B'],
    'BillAmount': [5000, 7000]
})
merged_data = pd.concat([merged_data, new_patients], ignore_index=True)
print("\nDataset after adding new patients:\n", merged_data.tail())


Dataset after adding new patients:
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   
5        101      NaN   Cardiology      Dr. A  5000.000000               NaN   
6        102      NaN    Neurology      Dr. B  7000.000000               NaN   

   FinalAmount  
2       5000.0  
3       3200.0  
4       4000.0  
5          NaN  
6          NaN  


# 9.Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [13]:
new_billing_cols = pd.DataFrame({
    'InsuranceCovered': [9200, 2300, 1400],
    'FinalAmount': [3300, 6000, 4500]
})
merged_data = pd.concat([merged_data.reset_index(drop=True), new_billing_cols], axis=1)
print("\nFinal dataset with new billing columns:\n", merged_data.head())


Final dataset with new billing columns:
    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000            2000.0   
1        102      Bob    Neurology   Dr. John  6233.333333            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   

   FinalAmount  InsuranceCovered  FinalAmount  
0       3000.0            9200.0       3300.0  
1       3500.0            2300.0       6000.0  
2       5000.0            1400.0       4500.0  
3       3200.0               NaN          NaN  
4       4000.0               NaN          NaN  


In [15]:
print("\nFinal cleaned dataset:", merged_data)


Final cleaned dataset:    PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000            2000.0   
1        102      Bob    Neurology   Dr. John  6233.333333            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith  6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose  6233.333333            1000.0   
5        101      NaN   Cardiology      Dr. A  5000.000000               NaN   
6        102      NaN    Neurology      Dr. B  7000.000000               NaN   

   FinalAmount  InsuranceCovered  FinalAmount  
0       3000.0            9200.0       3300.0  
1       3500.0            2300.0       6000.0  
2       5000.0            1400.0       4500.0  
3       3200.0               NaN          NaN  
4       4000.0               NaN          NaN  
5          NaN               NaN          NaN  